_This notebook requires Python 3_

# Inspecting aggregated impact outputs

Using _HazImp_ for evaluating the impacts of a severe weather event, the code automatically aggregates on the basis of a regional areas - either meshblock, SA1 or SA2. Other regions could also be considered. 

Currently, the process relies on specific formats for input and produces specific format output. 

### Input data

* Forecast data is translated into GeoTIFF format from the raw NetCDF files generated by the ACCESS model. The _HazImp_ code needs to be modified to directly read netCDF format. Issues here could include conflicts in GDAL, NetCDF and Python libraries used to execute the code.
* Exposure data is provided in a pre-defined comma-separated values format. _HazImp_ should be updated to enable ingestion of a service
* The forecast dataset is specified in a text-based configuration file. It is possible to provide a list of hazard layers to the configuration of _HazImp_, which would enable an ensemble of forecast data to be ingested. This has not been tested with the current development branch. Alternatives could include making the input forecast a command line argument, or scripting an intermediate process to edit the configuration file for each ensemble member realisation.

### Output data

* Output is currently a comma-separated values format, with entries for each region in the forecast model domain. Ideally this should be changed to generate a vector shapefile, or possibly webservice that can be ingested by end-user systems (e.g. Visual Weather can read shapefiles).


In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import os
from os.path import join as pjoin
import pandas as pd
import geopandas as gpd
import numpy as np

import seaborn as sns
sns.set_style('darkgrid')


The columns represent the mean and total values for each region. 

* "SLM" = mean structural loss
* "SLT" = total structural loss
* "RVM" = mean replacement value
* "RVT" = total replacement value
* "SLRM" = mean structural loss ratio ([0, 1])
* "SLRT" = standard deviation of structural loss ratio


In [ ]:
colnames = ["ID", "SA1_MAIN16", "SLM", "SLT", "RVM", "RVT", "SLRM", "SLRT"]

In [ ]:
impactPath = "N:/georisk/HaRIA_B_Wind/projects/impact_forecasting/data/impact"
impactFile = "agg-impact-wind-ens22-SA1.csv"
dtype = {"SA1_MAIN16":str}
df = pd.read_csv(pjoin(impactPath, impactFile), names=colnames, dtype=dtype,skiprows=3)

In [ ]:
ax = df['SLRM'].plot.hist()
ax.set_title("Mean structural loss ratio")
ax.set_xlabel("Structural loss ratio")

In [ ]:
regionPath = "N:/georisk/HaRIA_B_Wind/projects/impact_forecasting/data/exposure"
regionFile = "SA1_2016_AUST.shp"
gdf = gpd.read_file(pjoin(regionPath, regionFile))

In [ ]:
mgdf = gdf.merge(df, on="SA1_MAIN16")

In [ ]:
ax = mgdf.plot(column="SLRM", cmap='RdYlGn_r')


In [ ]:
mgdf.to_file(pjoin(impactPath,"agg-impact-wind-ens22-SA1.shp" ))